<a href="https://colab.research.google.com/github/laresbernardo/notebooks/blob/master/Comparing_US_Sectors_Stocks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#devtools::install_github("laresbernardo/lares")
library(lares)
library(dplyr)
library(ggplot2)

In [ ]:
symbols <-c("SPY", "XLV", "XLK", "XLE", "XLF", "XLC", "XLI", "XLY", "XLP", "XLRE", "XLU", "XLB")
from <- "2020-02-19"

In [ ]:
# Get the data
eod_sectors <- stocks_hist(symbols, from = from) %>%
  group_by(Symbol) %>%
  arrange(desc(Date)) %>%
  mutate(
    norm_close = Value / last(Value),
    type = if_else(Symbol == "SPY", "Market", "Sector"),
    sector = case_when(
      Symbol == "SPY"  ~ "S&P 500",
      Symbol == "XLB"  ~ "Materials",
      Symbol == "XLE"  ~ "Energy",
      Symbol == "XLU"  ~ "Utilities",
      Symbol == "XLI"  ~ "Industrial",
      Symbol == "XLRE" ~ "Real Estate",
      Symbol == "XLV"  ~ "Health",
      Symbol == "XLK"  ~ "Technology",
      Symbol == "XLF"  ~ "Financial",
      Symbol == "XLC"  ~ "Communication",
      Symbol == "XLY"  ~ "Consumer Discretionary",
      Symbol == "XLP"  ~ "Consumer Staples",
      TRUE ~ "Other")) %>% 
  ungroup()

In [ ]:
cols <- names(lares::lares_pal()$palette)
ord <- eod_sectors %>% filter(Date == max(Date)) %>% 
  arrange(desc(norm_close)) %>% 
  mutate(sector = autoline(sector, 1)) %>% .$sector
eod_sectors %>%
  mutate(
    sector = autoline(sector, 1),
    sector = factor(sector, levels = ord)) %>%
  ggplot(aes(x = Date, y = norm_close, colour = type)) +
  geom_rect(aes(xmin = min(Date), xmax = max(Date), ymin = -Inf, ymax = Inf), 
            fill = if_else(eod_sectors$type == "Market", cols[3], NULL), colour = "white") +
  geom_hline(yintercept = 1, linetype = "dashed", colour = "grey80") +
  geom_line(key_glyph = "timeseries") +
  facet_wrap(.~sector) +
  scale_colour_manual(values = cols[c(1, 4)]) +
  scale_y_percent() +
  labs(
    title = "S&P 500 Sector Impact of Covid-19",
    subtitle = glued("Relative to {from}"),
    x = NULL, y = NULL, colour = NULL) +
  theme(axis.text.x = element_text(angle = 45, hjust = 1)) +
  theme_lares(legend = "top")
  